# Camera Calibration - Finding Camera Matrix

In [1]:
# Import required modules 
import cv2 
import numpy as np 
import os 
import glob 

In [2]:
# Define the dimensions of checkerboard 
CHECKERBOARD = (6, 9) 

In [3]:
# stop the iteration when specified  accuracy, epsilon, is reached or specified number of iterations are completed. 
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [4]:
# for 3d points
temp = np.zeros((1, CHECKERBOARD[0]  * CHECKERBOARD[1],  3), np.float32) 
print(temp.shape)  # simply stores the 3d coordinate of each point of our checkerboard

(1, 54, 3)


In [8]:
# Vector for 3D points 
threedpoints = [] 
# Vector for 2D points 
twodpoints = [] 
  
#  3D points real world coordinates 
objectp3d = np.zeros((1, CHECKERBOARD[0]  * CHECKERBOARD[1],  3), np.float32) 
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],  0:CHECKERBOARD[1]].T.reshape(-1, 2) 
prev_img_shape = None
objectp3d  # this is our 3D checkerboard coordinate system 

array([[[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [5., 0., 0.],
        [0., 1., 0.],
        [1., 1., 0.],
        [2., 1., 0.],
        [3., 1., 0.],
        [4., 1., 0.],
        [5., 1., 0.],
        [0., 2., 0.],
        [1., 2., 0.],
        [2., 2., 0.],
        [3., 2., 0.],
        [4., 2., 0.],
        [5., 2., 0.],
        [0., 3., 0.],
        [1., 3., 0.],
        [2., 3., 0.],
        [3., 3., 0.],
        [4., 3., 0.],
        [5., 3., 0.],
        [0., 4., 0.],
        [1., 4., 0.],
        [2., 4., 0.],
        [3., 4., 0.],
        [4., 4., 0.],
        [5., 4., 0.],
        [0., 5., 0.],
        [1., 5., 0.],
        [2., 5., 0.],
        [3., 5., 0.],
        [4., 5., 0.],
        [5., 5., 0.],
        [0., 6., 0.],
        [1., 6., 0.],
        [2., 6., 0.],
        [3., 6., 0.],
        [4., 6., 0.],
        [5., 6., 0.],
        [0., 7., 0.],
        [1., 7., 0.],
        [2., 7., 0.],
        [3

In [9]:
import matplotlib.pyplot as plt

In [10]:
IMAGE_FOLDER  = r"C:\Users\Piyush\Desktop\Checkerboard - Images"
images = glob.glob(os.path.join(IMAGE_FOLDER,'*'))

In [11]:
#images = glob.glob(r'C:\Users\Piyush\Desktop\Checkerboard - Images') 
for filename in images: 
    image = cv2.imread(filename)
    print(image.shape)
    #plt.imshow(image)

(155, 200, 3)
(200, 156, 3)
(200, 156, 3)


In [12]:
# Extracting path of individual image stored 
# in a given directory. Since no path is 
# specified, it will take current directory 
# jpg files alone 
#images = glob.glob(r'C:\Users\Piyush\Desktop\Checkerboard - Images') 
  
for filename in images: 
    image = cv2.imread(filename) 
    #print(image.shape)
    grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Find the chess board corners 
    # If desired number of corners are found in the image then ret = true 
    ret, corners = cv2.findChessboardCorners(grayColor, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH  + cv2.CALIB_CB_FAST_CHECK + 
                                                                      cv2.CALIB_CB_NORMALIZE_IMAGE) 
  
    # If desired number of corners can be detected then, 
    # refine the pixel coordinates and display 
    # them on the images of checker board 
    if ret == True: 
        threedpoints.append(objectp3d) 
  
        # Refining pixel coordinates for given 2d points. 
        corners2 = cv2.cornerSubPix(grayColor, corners, (11, 11), (-1, -1), criteria) 
        #print(corners2.shape)
        #print(corners2)
        twodpoints.append(corners2) 
  
        # Draw and display the corners 
        image = cv2.drawChessboardCorners(image,  CHECKERBOARD,  corners2, ret) 
  
    cv2.imshow('img', image) 
    cv2.waitKey(0) 
  
cv2.destroyAllWindows() 
  
h, w = image.shape[:2] 
  


In [32]:
  
# Perform camera calibration by passing the value of above found out 3D points (threedpoints) and its corresponding pixel 
# coordinates of the  detected corners (twodpoints) 
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(threedpoints, twodpoints, grayColor.shape[::-1], None, None) 
  
# Displayig required output 
print(" Camera matrix:") 
print(matrix) 
 


 Camera matrix:
[[20.10654304  0.         84.16362263]
 [ 0.         20.34239482 95.42267081]
 [ 0.          0.          1.        ]]


In [31]:
print("\n Distortion coefficient:") 
print(distortion)


 Distortion coefficient:
[[-9.40501496e-04  3.73198946e-05 -2.32754445e-03  3.95213785e-04
  -6.01340412e-07]]


In [24]:
rotation_matrix = np.array(r_vecs)
print(rotation_matrix.shape)

(3, 3, 1)


In [27]:
rotation_matrix = rotation_matrix.reshape(3,3).T
rotation_matrix

array([[-0.04742568, -0.07882398, -0.01784273],
       [ 0.02932197, -0.00961833,  0.04617962],
       [ 1.50950267,  3.07805624, -0.07272072]])

In [28]:
translation_matrix = np.array(t_vecs)
print(translation_matrix.shape)

(3, 3, 1)


In [29]:
translation_matrix = translation_matrix.reshape(3,3).T
translation_matrix

array([[ 4.63365547,  2.32806935, -3.16534453],
       [-3.7646618 ,  3.99318851, -3.45998477],
       [ 1.35562517,  1.36446905,  1.38547247]])

# Main aim is to find Camera Matrix only!!!

### These transalation and rotational matrix are with respect to chessboard 2d and 3d coordinate system not with respect to our original GCP 3d and 2d coordinates